In [1]:
import os
import psycopg2
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
import joblib


conn = psycopg2.connect(
    dbname="mimiciv",
    user="uti_user",
    password="s0f4C1#4",
    host="localhost",
    port="5432"
)
print("Conexão ao banco estabelecida.")

Conexão ao banco estabelecida.


DETAIL:  The database was created using collation version 2.35, but the operating system provides version 2.39.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE mimiciv REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


In [2]:
cursor = conn.cursor()
cursor.execute("ROLLBACK")
cursor.execute("""
CREATE TEMP TABLE coorte_uti AS
WITH primeira_uti AS (
    SELECT
        i.subject_id,
        i.hadm_id,
        i.stay_id,
        i.intime,
        i.outtime,
        ROW_NUMBER() OVER (PARTITION BY i.hadm_id ORDER BY i.intime) AS rn
        -- Se quiser 1ª UTI da vida do paciente, troque acima por:
        -- ROW_NUMBER() OVER (PARTITION BY i.subject_id ORDER BY i.intime) AS rn
    FROM mimiciv_icu.icustays i
)
SELECT
    p.subject_id,
    p.hadm_id,
    p.stay_id,
    p.intime,
    p.outtime,
    EXTRACT(EPOCH FROM (p.outtime - p.intime)) / 3600.0 AS los_hours,
    /* idade aproximada na admissão hospitalar usando âncoras do MIMIC-IV */
    (pat.anchor_age + EXTRACT(YEAR FROM adm.admittime)::INT - pat.anchor_year) AS idade_admissao
FROM primeira_uti p
JOIN mimiciv_hosp.admissions adm ON p.hadm_id = adm.hadm_id
JOIN mimiciv_hosp.patients   pat ON p.subject_id = pat.subject_id
WHERE p.rn = 1
  AND (pat.anchor_age + EXTRACT(YEAR FROM adm.admittime)::INT - pat.anchor_year) >= 18   -- adultos
  AND (p.outtime - p.intime) >= INTERVAL '24 hours';                                      -- LOS ≥ 24h
""")
conn.commit()



df_coorte = pd.read_sql("SELECT COUNT(*) AS n_coorte FROM coorte_uti;", conn)
print("Total pacientes no coorte:", df_coorte['n_coorte'][0])


Total pacientes no coorte: 52028


/tmp/ipykernel_158969/1357890584.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_coorte = pd.read_sql("SELECT COUNT(*) AS n_coorte FROM coorte_uti;", conn)
